# Task 3: Helper notebook for loading the data and saving the predictions

In [1]:
import pickle
import gzip
import numpy as np
import os

### Helper functions

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [3]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

### Load data, make predictions and save prediction in correct format

In [4]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")

In [2]:
import keras 
import tensorflow as tf


# Define your class weights (replace these values with your actual class weights)
class_weights = [10.0, 1.0]  # For example, 3 classes with respective weights

# Define the custom loss function with weighted sparse categorical cross-entropy
@keras.saving.register_keras_serializable()
def weighted_sparse_binary_crossentropy(y_true, y_pred):
    # Calculate the regular SparseCategoricalCrossentropy loss
    scce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    loss = scce(y_true, y_pred)

    # Apply class weights to the loss
    weights = tf.gather(class_weights, y_true)
    weighted_loss = tf.reduce_mean(weights * loss)

    return weighted_loss

2023-12-18 12:43:40.010089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 12:43:40.010373: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 12:43:40.019314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 12:43:40.529163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import tensorflow as tf

# Load model
model = tf.keras.models.load_model("models/model2.keras")

In [ ]:
def get_prediction(height, width, small_image):
  #print(small_image.shape)
  small_image = small_image.reshape((1,) + small_image.shape)
  #print(small_image.shape)
  prediction = model.predict(small_image)
  #print(f'prediction shape {prediction.shape}')
  prediction = prediction[0]
  #print(f'prediction shape {prediction.shape}')
  prediction = Image.fromarray(prediction[:,:,0])
  prediction = prediction.resize((128, 128))
  return np.array(prediction, dtype=bool)

# make prediction for test
predictions = []
for d in test_data:
    prediction = np.array(np.zeros_like(d['video']), dtype=bool)


    video = d['video']
    height = video.shape[1]
    width = video.shape[2]

    # Resize the video so it fits into the model
    small_video = np.array([np.array(Image.fromarray(image).resize((128, 128))) for image in video])

    small_image = small_video[0].reshape((1,) + small_video[0].shape)

    prediction = model.predict(small_video)

    prediction = prediction[:,:,:,0]

    prediction = [np.array(Image.fromarray(mask).resize((width, height)), dtype=bool) for mask in prediction]


    prediction = np.array(prediction, dtype=bool)

    print(f"new predicition shape {prediction.shape} for id {d['name']}")

    print(prediction.shape)
    
    obj = {
        'name': d['name'],
        'prediction': prediction
        }
    
    save_zipped_pickle(obj, f"tmp/{d['name']}.pkl")

    # DATA Strucure
    predictions.append(obj)


In [ ]:
# save in correct format
save_zipped_pickle(predictions, 'my_predictions.pkl')